In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

Đọc dữ liệu từ file csv

In [5]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week_1/data/multinomial_nb_orders.csv')
print(df.head(5))

      cuisine     city device user_segment    promo fav_ingredient  \
0    american  city_16    web          new  promo_1          onion   
1        thai  city_10    ios       family  promo_7       cilantro   
2    american  city_18    ios      student  promo_1         cheese   
3    japanese  city_10    web       family  promo_3         ginger   
4  vietnamese  city_10    web       family     none     fish_sauce   

  spice_level day_of_week price_bucket  
0         hot         Wed         high  
1        mild         Wed          mid  
2         hot         Fri          mid  
3        mild         Tue         high  
4        mild         Wed          low  


Tính xác suất P(c) = Nc/N

In [6]:
Y = df['cuisine']
y_labels = np.zeros((len(set(Y)), 2), dtype=Y.dtype)
i = 0
for label in set(Y):
  y_labels[i, 0] = label
  y_labels[i, 1] = (Y == label).sum() / len(Y)
  i+=1
